# 使用 VLM 监控的 SmolVLA 移除红块 Agent

本笔记展示如何结合 `SmolVLA` policy 与 GPT 系列的视觉语言模型（VLM），在 MuJoCo 仿真中以 Agent 形式自动执行“移除红块”任务，并根据 VLM 的反馈动态切换策略。


## 1. 环境与日志初始化

确保已经安装本仓库推荐的 conda 环境并成功运行过 `SmolVLA` 的部署示例。以下单元将设置日志级别，方便在 Notebook 中观察 Agent 的行为。


In [1]:
import logging

logging.basicConfig(level=logging.INFO, format="[%(asctime)s] %(levelname)s - %(message)s")
logging.getLogger("agent.remove_block_agent").setLevel(logging.INFO)
print("✅ 日志初始化完成")


✅ 日志初始化完成


## 2. 配置 VLM 客户端与策略仓库

- `policy_repos` 用于注册不同相对位置的策略（可按需补充或替换）。
- VLM 默认处于 `dry_run` 模式，不会真实调用 GPT API。若要接入 OpenAI，请在外部配置 `OPENAI_API_KEY`，并将 `vlm_dry_run` 设为 `False`。


In [2]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("✅ 已将项目根目录加入 sys.path:", PROJECT_ROOT)


✅ 已将项目根目录加入 sys.path: /home/dragon/Agent_empty_plat


In [ ]:
from agent.remove_block_agent import (
    RemoveBlockAgentConfig,
    RemoveBlockAgent,
    GPTVLMClient,
)

policy_repos = {
    "center": "DragonHu/smolvla_UR52_empty_plat_block_fixed_v2",
    # 如需支持不同位置的策略，可在此添加：
    # "left": "<your-hf-repo-left>",
    # "right": "<your-hf-repo-right>",
    # "front": "<your-hf-repo-front>",
    # "back": "<your-hf-repo-back>",
}

config = RemoveBlockAgentConfig(
    policy_repos=policy_repos,
    default_policy="center",
    vlm_dry_run=True,  # 设置为 False 即可启用真实 GPT API
)
config.ensure()
print(config)


/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-11-02 21:40:10,242] INFO - PyTorch version 2.7.1 available.


RemoveBlockAgentConfig(xml_scene_path='./asset/scene_remove_block.xml', dataset_root='./demo_data_language', policy_repos={'center': 'DragonHu/smolvla_UR52_empty_plat_block_fixed_v2'}, default_policy='center', device='cuda', image_size=256, sim_steps_per_action=8, max_steps_per_attempt=180, render=True, log_interval=20, save_debug_images=False, debug_dir='./media/remove_block_agent', seed=None, vlm_model='gpt-4.1-mini', vlm_base_url='https://api.openai.com/v1', vlm_dry_run=True)


In [4]:
vlm_client = GPTVLMClient(
    model_name=config.vlm_model,
    base_url=config.vlm_base_url,
    dry_run=config.vlm_dry_run,
)

agent = RemoveBlockAgent(config=config, vlm_client=vlm_client)
print("✅ Agent 初始化完成")


[2025-11-02 21:40:10,816] INFO - GPTVLMClient operating in dry-run mode (no API calls will be made).


ValueError: mjParseXML: resource not found via provider or OS filesystem: '/home/dragon/Agent_empty_plat/agent/asset/scene_remove_block.xml'

## 3. 自定义 Prompt（可选）

`RemoveBlockAgent` 内部已经准备好默认的 prompt 模板。如果需要更细粒度的控制，可以继承 `GPTVLMClient` 或直接覆写 `describe_scene` / `evaluate_outcome` 来注入自定义 prompt。下面的代码块给出一个简单示例：


In [ ]:
class CustomVLMClient(GPTVLMClient):
    """示例：通过覆写方法定制 prompt 结构。"""

    def describe_scene(self, image_bytes: bytes, task_hint: str):
        # 可以在这里构造完全自定义的 prompt
        return super().describe_scene(image_bytes, task_hint)

    def evaluate_outcome(self, image_bytes: bytes, stage_hint: str):
        # 同理，这里也可以加入额外的 JSON 字段约束
        return super().evaluate_outcome(image_bytes, stage_hint)

# 如需启用自定义 prompt，只需用 CustomVLMClient 替换 GPTVLMClient 即可：
# vlm_client = CustomVLMClient(model_name=config.vlm_model, base_url=config.vlm_base_url, dry_run=config.vlm_dry_run)
# agent = RemoveBlockAgent(config=config, vlm_client=vlm_client)


## 4. 启动单轮 Agent

运行 `run_episode` 将：

1. 重置仿真并采样初始图像发送给 VLM，总结任务条件；
2. 根据 VLM 返回的 block 相对位置选择策略，并执行 SmolVLA policy；
3. 结束后再次上传图像给 VLM，由其判断成功与否，并给出失败原因；
4. 将每次尝试的详细信息记录在字典中返回。


In [ ]:
summary = agent.run_episode(
    instruction="Remove the red block from the plate.",
    max_attempts=2,
)
print("任务成功:", summary["success"])


### 查看完整日志

使用 `json.dumps` 可以更直观地观察每次尝试的 VLM 反馈、策略选择与执行步数。


In [ ]:
import json

print(json.dumps(summary, indent=2, ensure_ascii=False))


## 5. 后续可扩展方向

- **多策略扩展**：补充不同位置的 Hugging Face checkpoint，并在 `policy_repos` 中注册。
- **真实 VLM 接口**：设置 `OPENAI_API_KEY`，将 `vlm_dry_run=False`，即可通过 GPT 的多模态接口获取真实分析。
- **调试与可视化**：结合 `ipywidgets` 增加手动重试按钮，或在 `config.save_debug_images=True` 时自动保存关键帧。
- **终端控制**：`RemoveBlockAgent` 是纯 Python 类，可在脚本/终端中直接复用，实现批量评估或离线分析。
